<a href="https://colab.research.google.com/github/RituAnilkumar/EarthEngine/blob/master/Exercise1_single_band_visualization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1 align="center"> Google Earth Engine Python API Introductory Tutorial 1 </h1>

<p align="center"> **Author** : Ritu Anilkumar</p> 

<p align="center"> Updated on 14 January 2020 </p>

<p align="justify">This tutorial is an introduction to the Python API of Google Earth Engine. We can import the Earth Engine library like any other library using the import function. This makes most functions fairly similar to the JavaScript API. First we need to authenticate Earth Engine and initialize using our username and password. Please click the link that appears and grant the Earth Engine Authenticator to proceed with the tutorial. The provided link should be pasted into the space demarcated for the same.</p>

In [0]:
# Import the earth engine library and authenticate with your credentials
import ee
ee.Authenticate()
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/vQHWTJvojV6MCZiqZdednHO31NdmgJl9SfN_LKvOgyQpN34xXB3Q7ik

Successfully saved authorization token.


<p align="justify"> The best part about using Google Earth Engine is that we don't have to go through the hassle of downloading the data. Petabytes of data is stored in their Data Catagolue from Landsat series of sensors to Sentinel series, MODIS, SRTM, ALOS, ERA5 Climate Reananlysis, land surface temperature, night lights and so on. For an extensive list of datasources ingested into Google Earth Engine see [here](https://developers.google.com/earth-engine/datasets). In the next code snippet, we will see how we can access this data and perform some simple visualizations.</p>

<p align="justify"> Here, we have used the version 6 MODIS vegetation indices product: MYD13A1 V6 product. This provides a Vegetation Index (VI) value at a per pixel basis. There are two primary vegetation layers. The first is the Normalized Difference Vegetation Index (NDVI) and the second vegetation layer is the Enhanced Vegetation Index (EVI) that minimizes canopy background variations and maintains sensitivity over dense vegetation conditions. The EVI also uses the blue band to remove residual atmosphere contamination caused by smoke and sub-pixel thin cloud clouds. The MODIS NDVI and EVI products are computed from atmospherically corrected surface reflectances that have been masked for water, clouds, heavy aerosols, and cloud shadows. </p>

In [0]:
# Get the image collection using the datasets available
ImCol=ee.ImageCollection('MODIS/006/MYD13A1')
print(ImCol)

ee.ImageCollection({
  "type": "Invocation",
  "arguments": {
    "id": "MODIS/006/MYD13A1"
  },
  "functionName": "ImageCollection.load"
})


<p align="justify">We didn't get much meaningful information when we printed the ImCol. This is due to the way Python treats the Earth Engine objects. To extract meaningful information from the Earth Engine objects, we use the getInfo().</p>

In [0]:
print(ImCol.getInfo())

{'type': 'ImageCollection', 'bands': [], 'id': 'MODIS/006/MYD13A1', 'version': 1578921843230927, 'properties': {'system:visualization_0_min': '0.0', 'type_name': 'ImageCollection', 'thumb': 'https://mw1.google.com/ges/dd/images/MODIS_006_MOD13A1_thumb.png', 'description': '<p>The MYD13A1 V6 product provides a Vegetation Index (VI) value at a per pixel basis. There are two primary vegetation layers. The first is the Normalized Difference Vegetation Index (NDVI) which is referred to as the continuity index to the existing National Oceanic and Atmospheric Administration-Advanced Very High Resolution Radiometer (NOAA-AVHRR) derived NDVI. The second vegetation layer is the Enhanced Vegetation Index (EVI) that minimizes canopy background variations and maintains sensitivity over dense vegetation conditions. The EVI also uses the blue band to remove residual atmosphere contamination caused by smoke and sub-pixel thin cloud clouds. The MODIS NDVI and EVI products are computed from atmospherica

<p align="justify">If you want to check the properties of the images, we can use the Earth Engine first().</p>

In [0]:
print(ImCol.first().getInfo())

{'type': 'Image', 'bands': [{'id': 'NDVI', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': -32768, 'max': 32767}, 'dimensions': [86400, 43200], 'crs': 'SR-ORG:6974', 'crs_transform': [463.312716528, 0, -20015109.354, 0, -463.312716527, 10007554.677]}, {'id': 'EVI', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': -32768, 'max': 32767}, 'dimensions': [86400, 43200], 'crs': 'SR-ORG:6974', 'crs_transform': [463.312716528, 0, -20015109.354, 0, -463.312716527, 10007554.677]}, {'id': 'DetailedQA', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 65535}, 'dimensions': [86400, 43200], 'crs': 'SR-ORG:6974', 'crs_transform': [463.312716528, 0, -20015109.354, 0, -463.312716527, 10007554.677]}, {'id': 'sur_refl_b01', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': -32768, 'max': 32767}, 'dimensions': [86400, 43200], 'crs': 'SR-ORG:6974', 'crs_transform': [463.312716528, 0, -20015109.354, 0, -463.312716527, 10007554.677]}, {'id': 'sur_

<p align="justify">Here, first is a reducer that reduces the Image Collection to an image. There are several other reducers such as mean, median, mosaic available. Try them out yourself. We will use the qualityMosaic reducer. This reducer allows us to generate mosaics by checking each and every pixel value and creating an image of the largest values. In the example below, we use the NDVI band to generate the quality mosaic. This means that we will create one image from the image collection with each of the pixel values indicating the greenest pixel from the collection. This is called the greenest pixel image</p>

In [0]:
# Generating the greenest pixel image from the image collection using the quality mosaic reducer
im=ImCol.qualityMosaic('NDVI')

<p align="justify">Now, let's take a look at the properties of this image. We see that the image still contains several bands including NDVI, EVI, surface reflectance bands and quality bands. But how we can extract the NDVI image from this?</p>

In [0]:
# Extracting the NDVI band from the image
print(im.getInfo())
NDVI_im=im.select('NDVI')
print(NDVI_im.getInfo())

{'type': 'Image', 'bands': [{'id': 'NDVI', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': -32768, 'max': 32767}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'EVI', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': -32768, 'max': 32767}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'DetailedQA', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 65535}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'sur_refl_b01', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': -32768, 'max': 32767}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'sur_refl_b02', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': -32768, 'max': 32767}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'sur_refl_b03', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': -32768, 'max': 32767}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, 

<p align="justify">Now let's try to visualize the NDVI band.There are two ways to display the images in the Python API for Earth Engine. For colab projects, we can display as a thumbnail image or an interactive map. This cell depicts dislaying using thumbnails which uses the Image library from IPython.display. The visualization parameters need to be given just like we did in the GDAL tutorial. We have set the minimum and maximum values of 0 and 10000 respectively. I've set the image size to 512 and a custom palette was designed. Recall how we used viridis or magma or plasma or spectrum palette in Python's matplotlib.pyplot? Here, we define those using color codes.</p>

In [0]:
# Importing the Image lbrary to display the results
from IPython.display import Image
Image(url=NDVI_im.getThumbUrl({'min': 0, 'max': 10000, 'dimensions': 512, 'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5']}))

<p align="justify">Color codes used here are called hex codes. We can get the hex code of different numbers by simply searching for it. the first two digits refer to the red channel, the next two for the green channel and final two digits the blue channel. These are written as a hexanumeric combination. 00 implies 0 and ff implies the highest value. Therefore 000000 is black, ffffff is white, ff0000 is red, 00ff00 is blue, 0000ff is green.</p>
<p align="justify"> Let's consider another palette for a more detailed view. </p>

In [0]:
# Defining the palette in hexcodes and visualizing the image. Note, the dimensions have been changed from the previous as ademonstration.
paletteFull=['FFFFFF', 'CE7E45', 'DF923D', 'F1B555', 'FCD163', '99B718', '74A901','66A000', '529400', '3E8601', '207401', '056201', '004C00', '023B01','012E01', '011D01', '011301']
Image(url=NDVI_im.getThumbUrl({'min': 0, 'max': 10000, 'dimensions': 900, 'palette': paletteFull}))

 <p align="justify">In the above visualization, we are unable to interactively zoom in or zoom out and visualize regions of our interest. To do so, we need to use a platform called leaflet. Leaflet can be used with Python using a library called folium. Below, we configure folium to display the results on a leaflet base map. </p>

In [0]:
# First step is to import the folium library
import folium

# Define a function for displaying Earth Engine image tiles to folium map. 
def add_ee_layer(self, ee_image_object, vis_params, name):
  map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
  folium.raster_layers.TileLayer(
    tiles = map_id_dict['tile_fetcher'].url_format,
    attr = 'Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    name = name,
    overlay = True,
    control = True
  ).add_to(self)

# Add EE drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

# Create a folium map object where we will load our images and maps onto.
my_map = folium.Map(location=[24,121], zoom_start=3, height=500)

# Add the image to the map object in the style defined by the add_ee_layer function. The visualization parameters in curly braces can be replaced by a variable for readability
my_map.add_ee_layer(NDVI_im, {'min': 0,'max': 10000,'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5']}, 'NDVI palette 1')
my_map.add_ee_layer(NDVI_im, {'min': 0,'max': 10000,'palette': paletteFull}, 'NDVI palette 2')

# Adding layer controls
my_map.add_child(folium.LayerControl())

# Click on the map to get a pop up of latitude and longitude
folium.LatLngPopup().add_to(my_map)

# To display the map, use the display function and pass the map object as its argument
display(my_map)

<p align="justify">Yet another way exists to visualize the images and generate maps. This is if we use the Earth Engine plugin with QGIS. For this, please open QGIS 3.10 on your systems and go to Plugins>Manage and Install Plugins. Search for Google Earth Engine in the search bar and click install. During installation you will be required to authenticate access using your Google Earth Engine accounts. Please login to your gmail and authorize when requested to do so.</p>

<p align="justify">Next, ensure that the plugin is checked in the installed section and close. We can now open the Python console and write the Python code to acess the Earth Engine datasets and functionalities. For this, go to Pluings>Python Console. In the Console window, we can type the following commands </p>

In [0]:
# Importing the libraries of our interest
import ee
from ee_plugin import Map

# Get the image collection using the datasets available and filter based on location and date range
ImCol=ee.ImageCollection('MODIS/006/MYD13A1').filterDate(ee.Date('2018-01-01'),ee.Date('2018-12-31'))

# Generating the greenest pixel image from the image collection using the quality mosaic reducer
Im=ImCol.qualityMosaic('NDVI')

# Extract the NDVI band from the Image by using the select function with bandname passed as the argument
ImNDVI=Im.select('NDVI')

# Displaying the results on Earth Engine
Map.setCenter(91.17683945531849,26.17017283511727,5)
Map.addLayer(ImNDVI,{'palette': ['FFFFFF', 'CE7E45', 'DF923D', 'F1B555', 'FCD163', '99B718', '74A901','66A000', '529400', '3E8601', '207401', '056201', '004C00', '023B01','012E01', '011D01', '011301'], 'min': 0, 'max': 10000},'NDVI')